# Link Prediction

## Library Imports


In [ ]:
# === Imports ===
import os, sys, platform, importlib, math, random, json
import numpy as np, pandas as pd, torch, torch.nn as nn, torch.nn.functional as F
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import to_undirected
from torch_geometric.transforms import RandomLinkSplit
from scipy.sparse import csgraph
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from numpy.linalg import norm
from sklearn.metrics import roc_auc_score, average_precision_score

## Setup & Reproducibility


In [ ]:
# ===  Environment ===
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def _ver(m):
    try: return importlib.import_module(m).__version__
    except Exception: return "n/a"
print({
    "python": platform.python_version(),
    "numpy": _ver("numpy"),
    "pandas": _ver("pandas"),
    "torch": _ver("torch"),
    "torch_geometric": _ver("torch_geometric"),
    "sklearn": _ver("sklearn"),
    "scipy": _ver("scipy"),
    "networkx": _ver("networkx"),
})
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dependency Check (Optional)


In [ ]:
# Simple dependency report (optional)
import importlib, textwrap
for lib in ["networkx","numpy","pandas","torch","torch_geometric","matplotlib","sklearn","scipy","optuna","pyvis","IPython"]:
    try:
        __import__(lib)
        print(f"{lib}: OK")
    except Exception as e:
        print(f"{lib}: MISSING ({e})")


# Data and Network Setup


## Sociometric Network

In [ ]:
# Build/load sociometric graph so downstream cells have G_Sociometric

# Option A: from a CSV edgelist
# Expect a file 'sociometric_edges.csv' with columns 'u','v' (participant IDs)
# edges_df = pd.read_csv('sociometric_edges.csv', dtype=str)
# G_Sociometric = nx.from_pandas_edgelist(edges_df, source='u', target='v', create_using=nx.Graph())

# Option B: from an existing DataFrame in memory named `edges_df` (dtype=str, columns 'u','v')
# G_Sociometric = nx.from_pandas_edgelist(edges_df, source='u', target='v', create_using=nx.Graph())

if 'G_Sociometric' not in globals():
    raise RuntimeError("Define G_Sociometric (e.g., from sociometric_edges.csv) before proceeding.")

# Current graph stats (feature alignment occurs in Graph Construction cell)
print("Graph:", G_Sociometric.number_of_nodes(), "nodes;", G_Sociometric.number_of_edges(), "edges")


In [ ]:
# Load the CSV file
node_properties_all = pd.read_csv('baseline_features.csv', low_memory=False)

# Ensure the "record_id" column is treated as a string
node_properties_all["record_id"] = node_properties_all["record_id"].astype(str)

# Check the number of columns
num_columns = node_properties_all.shape[1]
print(f"Number of columns in the CSV file: {num_columns}")


In [ ]:
# === Preprocessing ===
# assumes node_properties_all (rows align to participant nodes) already loaded
id_cols = [c for c in node_properties_all.columns if c.lower() in {"record_id", "id", "node_id"}]
feats_df = node_properties_all.drop(columns=id_cols, errors="ignore")
miss = feats_df.isna().mean()
keep_cols = miss[miss <= 0.05].index.tolist()
X = feats_df[keep_cols]

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

num_pipe = Pipeline([("imp", SimpleImputer(strategy="mean")),
                     ("scaler", StandardScaler())])
cat_pipe = Pipeline([("imp", SimpleImputer(strategy="constant", fill_value="missing")),
                     ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True))])

preprocessor = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

X_mat = preprocessor.fit_transform(X)
X_np = (X_mat.toarray() if hasattr(X_mat, "toarray") else np.asarray(X_mat)).astype("float32")

In [ ]:
# === Graph Construction ===
# assumes an undirected NetworkX graph G_Sociometric already built

# Align graph nodes to available features (by 'record_id')
valid_nodes = set(node_properties_all['record_id'].astype(str))
G_Sociometric = G_Sociometric.subgraph(valid_nodes).copy()

# Build node list and edge index in aligned order
node_list = list(G_Sociometric.nodes())
node_to_idx = {n: i for i, n in enumerate(node_list)}
edges = np.array([(node_to_idx[u], node_to_idx[v]) for u, v in G_Sociometric.edges()], dtype=np.int64).T
edge_index = torch.tensor(edges, dtype=torch.long)
edge_index = to_undirected(edge_index, num_nodes=len(node_list))

# Re-index features to match node_list order
id_to_row = {rid: i for i, rid in enumerate(node_properties_all['record_id'].astype(str).tolist())}
row_idx = [id_to_row[n] for n in node_list]
X_aligned = X_np[row_idx]
x = torch.from_numpy(X_aligned).to(torch.float32)
assert x.shape[0] == len(node_list), 'Feature-node count mismatch after alignment'

data = Data(x=x, edge_index=edge_index, num_nodes=x.shape[0])
print(data)

In [ ]:
# === Link Splits ===
transform = RandomLinkSplit(
    num_val=0.15, num_test=0.15,
    is_undirected=True, split_labels=True,
    add_negative_train_samples=True, neg_sampling_ratio=1.0
)
train_data, val_data, test_data = transform(data)

# Encode val/test using only training adjacency
val_data.edge_index  = train_data.edge_index
test_data.edge_index = train_data.edge_index

train_data, val_data, test_data = train_data.to(device), val_data.to(device), test_data.to(device)

In [ ]:
# === GraphSAGE Model ===
class GraphSAGEModel(nn.Module):
    def __init__(self, in_dim: int, dropout: float = 0.5):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, 128); self.bn1 = nn.BatchNorm1d(128)
        self.conv2 = SAGEConv(128, 128);    self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = SAGEConv(128, 128)    
        self.dropout = nn.Dropout(dropout)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index); x = self.bn1(x); x = F.relu(x); x = self.dropout(x)
        x = self.conv2(x, edge_index); x = self.bn2(x); x = F.relu(x); x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x

    @staticmethod
    def decode(z, edge_index):
        src, dst = edge_index
        return (z[src] * z[dst]).sum(dim=-1)  

    @staticmethod
    def decode_all(z):  
        return torch.sigmoid(z @ z.T)

    def get_regularization_loss(self): 
        return 0.0

def _metrics(pos_logits, neg_logits):
    y_true = np.concatenate([np.ones_like(pos_logits.cpu()), np.zeros_like(neg_logits.cpu())])
    y_prob = torch.sigmoid(torch.cat([pos_logits, neg_logits])).detach().cpu().numpy()
    y_pred = (y_prob >= 0.5).astype(int)
    acc  = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    roc  = roc_auc_score(y_true, y_prob)
    ap   = average_precision_score(y_true, y_prob)  # PR-AUC
    return dict(acc=acc, prec=prec, rec=rec, f1=f1, roc_auc=roc, pr_auc=ap)

In [ ]:
# === Train/Eval ===
def _metrics(pos_logits, neg_logits):
    y_true = torch.cat([torch.ones_like(pos_logits), torch.zeros_like(neg_logits)]).cpu().numpy()
    y_prob = torch.sigmoid(torch.cat([pos_logits, neg_logits])).detach().cpu().numpy()
    y_pred = (y_prob >= 0.5).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    return dict(acc=acc, prec=prec, rec=rec, f1=f1)

def train_one_epoch(model, data, opt):
    model.train(); opt.zero_grad()
    z = model.encode(data.x, data.edge_index)
    pos = model.decode(z, data.pos_edge_label_index)
    neg = model.decode(z, data.neg_edge_label_index)
    loss = F.binary_cross_entropy_with_logits(
        torch.cat([pos, neg]),
        torch.cat([torch.ones_like(pos), torch.zeros_like(neg)])
    )
    loss.backward(); opt.step(); return float(loss)

@torch.no_grad()
def eval_split(model, data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    pos = model.decode(z, data.pos_edge_label_index)
    neg = model.decode(z, data.neg_edge_label_index)
    loss = F.binary_cross_entropy_with_logits(
        torch.cat([pos, neg]),
        torch.cat([torch.ones_like(pos), torch.zeros_like(neg)])
    )
    return float(loss), _metrics(pos, neg)

In [ ]:
# === HPO + Evaluation ===
HPO_MODE = "grid"  
MAX_EPOCHS = 200
PATIENCE   = 30
FIXED = dict(lr=3e-4, weight_decay=1e-5, dropout=0.5)
GRID  = {"lr":[3e-4, 1e-3], "dropout":[0.5], "weight_decay":[1e-5]}
N_TRIALS = 25  

def _fit(params):
    model = GraphSAGEModel(train_data.num_features, dropout=params["dropout"]).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=params["lr"], weight_decay=params["weight_decay"])
    best = {"vloss": float("inf"), "state": None}
    patience = PATIENCE
    for ep in range(1, MAX_EPOCHS+1):
        _ = train_one_epoch(model, train_data, opt)
        vloss, _vm = eval_split(model, val_data)
        if vloss < best["vloss"]:
            best["vloss"] = vloss
            best["state"] = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            patience = PATIENCE
        else:
            patience -= 1
            if patience == 0: break
    model.load_state_dict(best["state"]); return model

def select_params():
    if HPO_MODE == "none":
        return FIXED
    if HPO_MODE == "grid":
        import itertools
        best_params, best_f1, best_vloss = None, -1.0, float("inf")
        for lr, dp, wd in itertools.product(GRID["lr"], GRID["dropout"], GRID["weight_decay"]):
            params = {"lr": lr, "dropout": dp, "weight_decay": wd}
            m = _fit(params); vloss, vm = eval_split(m, val_data)
            if vm["f1"] > best_f1 or (vm["f1"] == best_f1 and vloss < best_vloss):
                best_params, best_f1, best_vloss = params, vm["f1"], vloss
        return best_params
    if HPO_MODE == "optuna":
        import optuna
        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=SEED))
        def objective(trial):
            params = {
                "lr": trial.suggest_float("lr", 1e-4, 5e-3, log=True),
                "dropout": trial.suggest_float("dropout", 0.30, 0.60),
                "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
            }
            m = _fit(params); _, vm = eval_split(m, val_data); return vm["f1"]
        study.optimize(objective, n_trials=N_TRIALS)
        return study.best_params
    raise ValueError(HPO_MODE)

best_params = select_params()
with open("best_params.json","w") as f: json.dump(best_params, f, indent=2)

TEST_SEEDS = [13,17,19,23,29]
rows = []
for s in TEST_SEEDS:
    # Set RNG seeds per iteration
    random.seed(s); np.random.seed(s); torch.manual_seed(s)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(s)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    # Re-split data per seed
    transform = RandomLinkSplit(
        num_val=0.15, num_test=0.15,
        is_undirected=True, split_labels=True,
        add_negative_train_samples=True, neg_sampling_ratio=1.0
    )
    train_data, val_data, test_data = transform(data)
    # Encode val/test using only training adjacency
    val_data.edge_index  = train_data.edge_index
    test_data.edge_index = train_data.edge_index
    train_data, val_data, test_data = train_data.to(device), val_data.to(device), test_data.to(device)

    # Fit and evaluate (metrics at fixed threshold 0.5)
    model = _fit(best_params)
    tloss, tm = eval_split(model, test_data)

    # Record thresholds for reproducibility
    tau_val = threshold_by_validation_f1(model, train_data, val_data)
    prob = score_all_pairs_matrix(model, train_data.edge_index, data.x.to(device))
    G_emp = edge_index_to_nx(test_data.num_nodes, test_data.pos_edge_label_index)
    tau_den = threshold_by_density(prob, target_edges=G_emp.number_of_edges())

    rows.append(dict(seed=s, loss=tloss, tau_val_f1=tau_val, tau_density=tau_den, **tm))
    torch.save(model.state_dict(), f"best_model_seed{s}.pt")

df = pd.DataFrame(rows)
display(df); print(df.agg(["mean","std"]).T)
df.to_csv("test_metrics_by_seed.csv", index=False)

In [ ]:
# === Spectral Similarity ===
def _lcc(G: nx.Graph):
    if G.number_of_nodes() == 0: return G
    return G.subgraph(max(nx.connected_components(G), key=len)).copy()

def _norm_lap_eigs(G: nx.Graph, k=None):
    A = nx.to_scipy_sparse_array(G, format="csr", dtype=float)
    L = csgraph.laplacian(A, normed=True)
    n = G.number_of_nodes()
    if n == 0: return np.array([])
    if k is None or k >= n-1:
        w = np.linalg.eigvalsh(L.toarray())
    else:
        w = eigsh(L, k=k, which="SM", return_eigenvectors=False)
    return np.sort(np.real(w))

def spectral_similarity(G_ref: nx.Graph, G_est: nx.Graph, k=None):
    e1 = _norm_lap_eigs(_lcc(G_ref), k); e2 = _norm_lap_eigs(_lcc(G_est), k)
    if len(e1) == 0 or len(e2) == 0: return np.nan
    m = max(len(e1), len(e2))
    e1 = np.pad(e1, (0, m-len(e1))); e2 = np.pad(e2, (0, m-len(e2)))
    return float(np.dot(e1, e2) / (norm(e1) * norm(e2)))

def structural_summary(G: nx.Graph):
    Gc = _lcc(G)
    deg = np.array([d for _, d in Gc.degree()])
    return dict(
        density = nx.density(G),
        avg_path_len = nx.average_shortest_path_length(Gc) if Gc.number_of_nodes() > 1 else np.nan,
        diameter = nx.diameter(Gc) if Gc.number_of_nodes() > 1 else np.nan,
        transitivity = nx.transitivity(G),
        deg_p01 = np.percentile(deg, 1) if deg.size else np.nan,
        deg_p25 = np.percentile(deg, 25) if deg.size else np.nan,
        deg_p50 = np.percentile(deg, 50) if deg.size else np.nan,
        deg_p75 = np.percentile(deg, 75) if deg.size else np.nan,
        deg_p99 = np.percentile(deg, 99) if deg.size else np.nan,
    )

In [ ]:
# === Helpers ===
@torch.no_grad()
def score_all_pairs_matrix(model, base_edge_index, x):
    model.eval()
    z = model.encode(x, base_edge_index)  # encode with training adjacency (leakage-safe)
    logits = z @ z.T
    prob = torch.sigmoid(logits)
    prob.fill_diagonal_(0)
    return prob  # NxN

def threshold_by_validation_f1(model, train_data, val_data):
    model.eval()
    with torch.no_grad():
        z = model.encode(val_data.x, train_data.edge_index)
        pos = model.decode(z, val_data.pos_edge_label_index)
        neg = model.decode(z, val_data.neg_edge_label_index)
        y_true = torch.cat([torch.ones_like(pos), torch.zeros_like(neg)]).cpu().numpy()
        y_prob = torch.sigmoid(torch.cat([pos, neg])).cpu().numpy()
        taus = np.linspace(0.01, 0.99, 99)
        f1s = [f1_score(y_true, (y_prob >= t).astype(int)) for t in taus]
        return float(taus[int(np.argmax(f1s))])

def threshold_by_density(prob_adj, target_edges: int):
    triu = prob_adj.triu(1).flatten().cpu().numpy()
    kth = max(1, int(target_edges))
    τ = np.partition(triu, -kth)[-kth]
    return float(τ)

def edge_index_to_nx(num_nodes, edge_index):
    G = nx.Graph(); G.add_nodes_from(range(num_nodes))
    ei = edge_index.detach().cpu().numpy()
    G.add_edges_from(map(tuple, ei.T))
    return G

def impute_graph_from_prob(prob_adj, τ: float):
    keep = (prob_adj >= τ).nonzero(as_tuple=False).cpu().numpy()
    G = nx.Graph(); G.add_nodes_from(range(prob_adj.shape[0])); G.add_edges_from(map(tuple, keep))
    return G

In [ ]:
# === External validation ===
# Expected files: file.csv (with 'record_id'), file_edges.csv (u,v as strings)

post_X = pd.read_csv('file_features.csv', low_memory=False)
post_X['record_id'] = post_X['record_id'].astype(str)

# Reuse the same fitted preprocessor (trained on baseline X)
post_feats = post_X.drop(columns=[c for c in post_X.columns if c.lower() in {'record_id','id','node_id'}], errors='ignore')
post_mat = preprocessor.transform(post_feats)
post_np  = (post_mat.toarray() if hasattr(post_mat, "toarray") else np.asarray(post_mat)).astype("float32")
x_new    = torch.from_numpy(post_np).to(torch.float32).to(device)

# Impute with attributes only (no edges)
τ = tau_val  # or tau_den; report both
G_imp_post = external_validate(model, x_new, τ=τ)

# Build empirical post-COVID graph
post_edges = pd.read_csv('file_edges.csv', dtype=str)
G_post_emp = nx.from_pandas_edgelist(post_edges, 'u', 'v', create_using=nx.Graph())

# Report structural similarity
print("Spectral similarity (post empirical vs imputed):", spectral_similarity(G_post_emp, G_imp_post))

In [ ]:
def self_loops_edge_index(n, device):
    idx = torch.arange(n, device=device)
    return torch.stack([idx, idx], dim=0)

@torch.no_grad()
def external_validate(model, x_new, τ: float = 0.5):
    base_adj = self_loops_edge_index(x_new.size(0), device)
    prob = score_all_pairs_matrix(model, base_adj, x_new)
    return impute_graph_from_prob(prob, τ)

In [ ]:
# Load one seed’s model and best params
with open("best_params.json") as f:
    bp = json.load(f)
model = GraphSAGEModel(train_data.num_features, dropout=bp.get("dropout", 0.5)).to(device)
model.load_state_dict(torch.load("best_model_seed13.pt", map_location=device))

# Thresholds for two operating points
tau_val = threshold_by_validation_f1(model, train_data, val_data)
prob = score_all_pairs_matrix(model, train_data.edge_index, data.x.to(device))
G_emp = edge_index_to_nx(data.num_nodes, data.edge_index) 
tau_den = threshold_by_density(prob, target_edges=G_emp.number_of_edges())

print("Threshold (val-F1):", tau_val)
print("Threshold (density-matched):", tau_den)

# Impute graphs at both thresholds
G_imp_val = impute_graph_from_prob(prob, tau_val)
G_imp_den = impute_graph_from_prob(prob, tau_den)
# Default selection for downstream cells (e.g., plots)
G_imp = G_imp_val

print("Spectral similarity (empirical vs imputed, val-F1):", spectral_similarity(G_emp, G_imp_val))
print("Spectral similarity (empirical vs imputed, density):", spectral_similarity(G_emp, G_imp_den))

print("Empirical summary:", structural_summary(G_emp))
print("Imputed summary (val-F1):", structural_summary(G_imp_val))
print("Imputed summary (density):", structural_summary(G_imp_den))

## Hyperparameter Optimization


In [ ]:
# Runner: Hyperparameter Optimization
import json
print("HPO_MODE =", HPO_MODE)

# Only select if not already computed
if 'best_params' not in globals():
    best_params = select_params()
    with open("best_params.json", "w") as f:
        json.dump(best_params, f, indent=2)

print("Selected best_params:", best_params)


## Specify Model Hyperparameters


In [ ]:
# Specify / confirm model hyperparameters for downstream cells
# Default to tuned best_params if available; otherwise use manuscript defaults.
params = dict(
    lr=best_params.get("lr", 3e-4) if 'best_params' in globals() else 3e-4,
    dropout=best_params.get("dropout", 0.5) if 'best_params' in globals() else 0.5,
    weight_decay=best_params.get("weight_decay", 1e-5) if 'best_params' in globals() else 1e-5,
)
print("Using params:", params)

# Optional manual override example:
# params.update({"lr": 3e-4, "dropout": 0.5, "weight_decay": 1e-5})


## Load or Train a Model


In [ ]:
# Load a trained model checkpoint (or train one if missing)
import os, torch

SEED_TO_LOAD = 13  # change to 17, 19, 23, 29 to load other seeds
ckpt = f"best_model_seed{SEED_TO_LOAD}.pt"

model = GraphSAGEModel(train_data.num_features, dropout=params["dropout"]).to(device)
if os.path.exists(ckpt):
    model.load_state_dict(torch.load(ckpt, map_location=device))
    print(f"Loaded checkpoint: {ckpt}")
else:
    print(f"Checkpoint {ckpt} not found; training a model with current params...")
    model = _fit(params)
    torch.save(model.state_dict(), ckpt)
    print(f"Saved new checkpoint: {ckpt}")

# Quick sanity: evaluate on validation and test
vloss, vm = eval_split(model, val_data)
tloss, tm = eval_split(model, test_data)
print("Val:", vm, "loss=", vloss)
print("Test:", tm, "loss=", tloss)


# Imputation and Visualization


The code below is designed to impute an undirected graph.

## Network Visualization


In [ ]:
# Visualize original test network
def live_plot(cur_G, name):
    """Plot network visualization"""
    from pyvis.network import Network
    import IPython
    net = Network(height="800px", width="100%", notebook=True, cdn_resources="remote")
    net.from_nx(cur_G)
    plotname = name + ".html"
    net.toggle_physics(True)
    net.force_atlas_2based(gravity=-80,central_gravity=0.005,spring_length=100,spring_strength=2,damping=0.1,overlap=1)
    net.show(plotname)
    return IPython.display.HTML(filename=plotname)


# Evaluation, Spectral Similarity, and Clean Imputation


In [ ]:
# Export per-seed test metrics to CSV (reconstruct if df missing)
import os, json
import pandas as pd

seeds_default = [13, 17, 19, 23, 29]

def _load_best_params():
    if os.path.exists('best_params.json'):
        with open('best_params.json') as f:
            return json.load(f)
    return {'dropout': 0.5, 'lr': 3e-4, 'weight_decay': 1e-5}

params = _load_best_params()

# Rebuild df if not present
if 'df' not in globals():
    rows = []
    for s in seeds_default:
        ckpt = f'best_model_seed{s}.pt'
        if not os.path.exists(ckpt):
            continue
        # Reconstruct the split and RNG state for this seed
        random.seed(s); np.random.seed(s); torch.manual_seed(s)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(s)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        transform = RandomLinkSplit(
            num_val=0.15, num_test=0.15,
            is_undirected=True, split_labels=True,
            add_negative_train_samples=True, neg_sampling_ratio=1.0
        )
        train_data, val_data, test_data = transform(data)
        val_data.edge_index  = train_data.edge_index
        test_data.edge_index = train_data.edge_index
        train_data, val_data, test_data = train_data.to(device), val_data.to(device), test_data.to(device)
        m = GraphSAGEModel(train_data.num_features, dropout=params.get('dropout', 0.5)).to(device)
        m.load_state_dict(torch.load(ckpt, map_location=device))
        tloss, tm = eval_split(m, test_data)
        tau_val = threshold_by_validation_f1(m, train_data, val_data)
        prob = score_all_pairs_matrix(m, train_data.edge_index, data.x.to(device))
        G_emp = edge_index_to_nx(data.num_nodes, data.edge_index) 
        tau_den = threshold_by_density(prob, target_edges=G_emp.number_of_edges())
        rows.append(dict(seed=s, loss=tloss, tau_val_f1=tau_val, tau_density=tau_den, **tm))
    df = pd.DataFrame(rows)

if 'df' in globals() and not df.empty:
    df.to_csv('test_metrics_by_seed.csv', index=False)
    print('Wrote test_metrics_by_seed.csv with', len(df), 'rows')
else:
    print('Warning: no seed metrics available to write; run the HPO/eval cell first.')


In [ ]:
# Degree distribution histogram for the imputed graph
import matplotlib.pyplot as plt
degs = [d for _, d in G_imp.degree()]
plt.figure(); plt.hist(degs, bins=30); plt.xlabel('Degree'); plt.ylabel('Count'); plt.title('Imputed graph degree distribution'); plt.show()
